In [1]:
import os
import pandas as pd

## load data from combined csv created from 'Barchart_combinorator.py'

In [2]:
final_combined_data = pd.read_csv('BarchartFutures/final_combined.csv')

In [3]:
final_combined_data['Time'] = pd.to_datetime(final_combined_data['Time'], errors='coerce')
final_combined_data.rename(columns={'Time': 'Date'}, inplace=True)
final_combined_data['day_of_week'] = final_combined_data['Date'].dt.weekday

In [4]:
grouped = final_combined_data.groupby('Futures_Contract_Type')

# Step 4: Loop through each group (Futures_Contract_Type) and check for missing Wednesdays
for contract_type, group in grouped:
    # Filter for Wednesdays (day_of_week == 2) in the group
    wednesday_data = group[group['day_of_week'] == 2]
    
    # Generate the full range of Wednesdays within the date range of this contract type
    date_range = pd.date_range(start=group['Date'].min(), end=group['Date'].max(), freq='W-WED')
    
    # Compare expected number of Wednesdays with actual number of Wednesdays
    expected_wednesdays = len(date_range)
    actual_wednesdays = len(wednesday_data)

    # Check for missing Wednesdays
    if expected_wednesdays != actual_wednesdays:
        missing_wednesdays = date_range[~date_range.isin(wednesday_data['Date'])]
        print(f"Futures_Contract_Type: {contract_type}")
        print(f"Missing Wednesday data points: {missing_wednesdays}")
        
        # Step 5: Loop through each missing Wednesday and find the surrounding Tuesday and Thursday
        for missing_wed in missing_wednesdays:
            # Find the corresponding Tuesday and Thursday
            tuesday = missing_wed - pd.Timedelta(days=1)
            thursday = missing_wed + pd.Timedelta(days=1)

            # Get the rows for Tuesday and Thursday (if they exist in the data)
            tuesday_data = group[group['Date'] == tuesday]
            thursday_data = group[group['Date'] == thursday]

            if not tuesday_data.empty and not thursday_data.empty:
                # Select only numeric columns for averaging
                numeric_columns = group.select_dtypes(include=['float64', 'int64']).columns
                # print(numeric_columns)
                avg_data = (tuesday_data[numeric_columns].iloc[0] + thursday_data[numeric_columns].iloc[0]) / 2

                # Create a new row with the calculated average
                new_row = pd.Series(avg_data)
            
                # Set the missing Wednesday's date and day_of_week in the new row
                new_row['Date'] = missing_wed
                new_row['Futures_Contract_Type'] = contract_type
                new_row['day_of_week'] = 2  # Set day_of_week to Wednesday
                new_row = new_row.reindex(final_combined_data.columns)
                # Append the interpolated data to the original dataframe
                final_combined_data = pd.concat([final_combined_data, new_row.to_frame().T], ignore_index=True)
                print(f'interpolated data added for {missing_wed}')
            else:
                print(f"Tuesday or Thursday missing for: {missing_wed}")
    else:
        print(f"Futures_Contract_Type: {contract_type} - No missing Wednesday data points.")

# Sort the final dataframe to keep it in order
final_combined_data.sort_values(by=['Futures_Contract_Type', 'Date'], inplace=True)


Futures_Contract_Type: KEH05
Missing Wednesday data points: DatetimeIndex(['2004-12-01', '2004-12-08'], dtype='datetime64[ns]', freq='W-WED')
Tuesday or Thursday missing for: 2004-12-01 00:00:00
Tuesday or Thursday missing for: 2004-12-08 00:00:00
Futures_Contract_Type: KEH06 - No missing Wednesday data points.
Futures_Contract_Type: KEH07 - No missing Wednesday data points.
Futures_Contract_Type: KEH08
Missing Wednesday data points: DatetimeIndex(['2007-07-04'], dtype='datetime64[ns]', freq='W-WED')
interpolated data added for 2007-07-04 00:00:00
Futures_Contract_Type: KEH09
Missing Wednesday data points: DatetimeIndex(['2007-07-04'], dtype='datetime64[ns]', freq='W-WED')
interpolated data added for 2007-07-04 00:00:00
Futures_Contract_Type: KEH10 - No missing Wednesday data points.
Futures_Contract_Type: KEH11 - No missing Wednesday data points.
Futures_Contract_Type: KEH12 - No missing Wednesday data points.
Futures_Contract_Type: KEH13
Missing Wednesday data points: DatetimeIndex([

In [5]:
barchart_wednesdays = final_combined_data.loc[(final_combined_data['day_of_week'] == 2)].reset_index(drop=True)
barchart_wednesdays

,Futures_Contract_Type,Date,Open,High,Low,Last,Change,%Chg,Volume,Open Int,day_of_week
0,KEH05,2003-11-12 00:00:00,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0
1,KEH05,2003-11-19 00:00:00,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0
2,KEH05,2003-11-26 00:00:00,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0
3,KEH05,2003-12-03 00:00:00,390.0,390.0,390.0,390.0,5.0,+1.30%,0.0,1.0,2.0
4,KEH05,2003-12-10 00:00:00,393.0,393.0,383.0,393.0,0.0,0.00%,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
11908,KEZ26,2024-08-07 00:00:00,636.75,636.75,636.75,636.75,-0.75,-0.12%,0.0,0.0,2.0
11909,KEZ26,2024-08-14 00:00:00,628.0,628.0,628.0,628.0,-0.75,-0.12%,0.0,0.0,2.0
11910,KEZ26,2024-08-21 00:00:00,625.25,625.25,625.25,625.25,-8.25,-1.30%,0.0,0.0,2.0
11911,KEZ26,2024-08-28 00:00:00,627.25,627.25,627.25,627.25,10.25,+1.66%,0.0,0.0,2.0


In [6]:
contract_month_mapping = {
    'KEU': 9,  # September
    'KEZ': 12, # December
    'KEH': 3,  # March
    'KEK': 5,  # May
    'KEN': 7   # July
}

# Convert the 'Date' column to datetime if it's not already
barchart_wednesdays['Date'] = pd.to_datetime(barchart_wednesdays['Date'])

# Function to get the month and year from a Futures_Contract_Type
def get_contract_month_and_year(futures_contract_type):
    # Extract the prefix (first 3 characters) and year (last 2 characters)
    prefix = futures_contract_type[:3]
    year = '20' + futures_contract_type[3:]  # Assume year is 20XX

    # Get the month from the mapping
    month = contract_month_mapping.get(prefix)
    return int(month), int(year)
barchart_wednesdays[['contract_month','contract_year']] = barchart_wednesdays['Futures_Contract_Type'].apply(lambda x: pd.Series(get_contract_month_and_year(x)))
# Filter the DataFrame: Exclude rows where the contract is "up"
def exclude_up_contracts(row):
    futures_contract_type = row['Futures_Contract_Type']
    contract_month, contract_year = get_contract_month_and_year(futures_contract_type)

    # Check if the Date matches the contract month and year
    return not ((row['Date'].month == contract_month) & (row['Date'].year == contract_year))

# Apply the filter to exclude rows where contracts are "up"
barchart_wednesdays_filtered = barchart_wednesdays[barchart_wednesdays.apply(exclude_up_contracts, axis=1)]
barchart_wednesdays_filtered

,Futures_Contract_Type,Date,Open,High,Low,Last,Change,%Chg,Volume,Open Int,day_of_week,contract_month,contract_year
0,KEH05,2003-11-12,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0,3,2005
1,KEH05,2003-11-19,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0,3,2005
2,KEH05,2003-11-26,374.0,374.0,374.0,374.0,0.0,0.00%,0.0,1.0,2.0,3,2005
3,KEH05,2003-12-03,390.0,390.0,390.0,390.0,5.0,+1.30%,0.0,1.0,2.0,3,2005
4,KEH05,2003-12-10,393.0,393.0,383.0,393.0,0.0,0.00%,0.0,1.0,2.0,3,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11908,KEZ26,2024-08-07,636.75,636.75,636.75,636.75,-0.75,-0.12%,0.0,0.0,2.0,12,2026
11909,KEZ26,2024-08-14,628.0,628.0,628.0,628.0,-0.75,-0.12%,0.0,0.0,2.0,12,2026
11910,KEZ26,2024-08-21,625.25,625.25,625.25,625.25,-8.25,-1.30%,0.0,0.0,2.0,12,2026
11911,KEZ26,2024-08-28,627.25,627.25,627.25,627.25,10.25,+1.66%,0.0,0.0,2.0,12,2026


In [7]:
barchart_wednesdays_filtered.to_csv('BarchartFutures/barchart_wednesdays.csv', index=False)